In [ ]:
!pip install netCDF4
from google.colab import drive
drive.mount('/content/gdrive')
import netCDF4 as nc
import numpy as np
import seaborn as sns
import pandas as pd
from shapely.geometry import Polygon
import shapely
import matplotlib.pyplot as plt
import os
from datetime import datetime
from tqdm.auto import tqdm
import xarray as xr
from sklearn.preprocessing import MinMaxScaler
from sklearn_pandas import DataFrameMapper, cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import joblib


import keras
from keras.models import Sequential, load_model
from keras.layers import Activation, Dense, Dropout, Input, BatchNormalization
from keras.objectives import MSE, MAE
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
import gc
import sys

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

**Start here for preprocessing grid**

In [ ]:
predictors = pd.read_csv('/content/gdrive/MyDrive/rainfall-forecast/merge_model/populate_grid/predictor_list.csv').drop(columns='Unnamed: 0').values.tolist()
predictors = list(np.concatenate(predictors).flat)

In [ ]:
keys_dict = dict.fromkeys(predictors, "In stock")
keys_dict['divergence'] = 'd'
keys_dict['fraction_of_cloud_cover'] = 'cc'
keys_dict['geopotential'] = 'z'
keys_dict['ozone_mass_mixing_ratio'] = 'o3'
keys_dict['potential_vorticity'] = 'pv'
keys_dict['relative_humidity'] = 'r'
keys_dict['specific_cloud_ice_water_content'] = 'ciwc'
keys_dict['specific_cloud_liquid_water_content'] = 'clwc'
keys_dict['specific_humidity'] = 'q'
keys_dict['specific_snow_water_content'] = 'cswc'
keys_dict['temperature'] = 't'
keys_dict['u_component_of_wind'] = 'u'
keys_dict['v_component_of_wind'] = 'v'
keys_dict['vertical_velocity'] = 'w'
keys_dict['vorticity'] = 'vo'

In [ ]:
latitude_values = xr.open_dataset('/content/gdrive/MyDrive/rainfall-forecast/merge_model/populate_grid/ERA5/pressure_levels/regridded/divergence.nc').latitude
longitude_values = xr.open_dataset('/content/gdrive/MyDrive/rainfall-forecast/merge_model/populate_grid/ERA5/pressure_levels/regridded/divergence.nc').longitude
time_values = xr.open_dataset('/content/gdrive/MyDrive/rainfall-forecast/merge_model/populate_grid/ERA5/pressure_levels/regridded/divergence.nc').time
level_values = xr.open_dataset('/content/gdrive/MyDrive/rainfall-forecast/merge_model/populate_grid/ERA5/pressure_levels/regridded/divergence.nc').level

In [ ]:
#array = np.empty((1827, 20, 285, 245))

lon = np.arange(245)
lat = np.arange(285)
time_interval = 200
time = np.arange(time_interval, 1827, time_interval)
np.append(time, 1827)

#for longitude in lon:
#  for latitude in lat:
date_range = pd.date_range('2016-01-01', '2020-12-31')
#date_fname_list = date_range.strftime('%Y%m%d').to_list()
#date_row_list = date_range.strftime('%Y-%m-%d').to_list()
list1 = []

range1 = 0
for predictor in predictors:
  #time2 = 0
  df = np.ma.getdata(nc.Dataset('/content/gdrive/MyDrive/rainfall-forecast/merge_model/populate_grid/ERA5/pressure_levels/regridded/' + predictor + '.nc')[keys_dict[predictor]][:, :, :, :])
   # print(time1)
  scaler = joblib.load('/content/gdrive/MyDrive/rainfall-forecast/merge_model/populate_grid/scalers/' + predictor + '_scaler.gz')
  for latitude in lat:
    for longitude in lon:
      df[:, :, latitude, longitude] = scaler.transform(df[:, :, latitude, longitude])
  #np.save(('/content/gdrive/MyDrive/rainfall-forecast/merge_model/populate_grid/ERA5/pressure_levels_prediction_preprocess/' + predictor), df)
  #list_values.append(df)
  #df_1 = np.concatenate(list_values, axis = 1)
  print(df.shape)
  #list1.append(df)
    #array[time2:time1, range1:(range1+20), :, :] = df
    #time2 += time_interval
  #del df
  df_nc = xr.DataArray(df, coords=[('time', date_range), ('lvl', level_values), ('lat', latitude_values), ('lon', longitude_values)])
  df_nc.to_netcdf('/content/gdrive/MyDrive/rainfall-forecast/merge_model/populate_grid/ERA5/pressure_levels_prediction_preprocess/' + predictor + '.nc')
  del df
  del df_nc
  del scaler
  gc.collect()
  #print(df_1.shape)
  #range1 += 20
  #print(date)


(1827, 20, 285, 245)
(1827, 20, 285, 245)
(1827, 20, 285, 245)
(1827, 20, 285, 245)
(1827, 20, 285, 245)
(1827, 20, 285, 245)
(1827, 20, 285, 245)
(1827, 20, 285, 245)
(1827, 20, 285, 245)
(1827, 20, 285, 245)
(1827, 20, 285, 245)
(1827, 20, 285, 245)
(1827, 20, 285, 245)
(1827, 20, 285, 245)
(1827, 20, 285, 245)


**Start here to use MLP to make predictions baesd on pressure variables**

In [2]:
model = load_model('/content/gdrive/MyDrive/rainfall-forecast/merge_model/populate_grid')

In [3]:
predictor_names = os.listdir('/content/gdrive/MyDrive/rainfall-forecast/merge_model/populate_grid/ERA5/pressure_levels_prediction_preprocess')

In [ ]:
#dataframe = np.empty((1827, 285, 245, 1))

In [4]:
dataframe = np.empty((827, 285, 245, 1))

In [5]:
time_range = np.arange(0, 1827, 100)
#time_range = np.arange(1000, 1827, 100)
lon = np.arange(245)
lat = np.arange(285)

for time in tqdm(time_range):
  actual_predictor = []
  time_delta = 100
  if time == 1800:
    time_delta = 26
  for predictor in predictor_names:
    file1 = np.ma.getdata(nc.Dataset('/content/gdrive/MyDrive/rainfall-forecast/merge_model/populate_grid/ERA5/pressure_levels_prediction_preprocess/' + predictor)['__xarray_dataarray_variable__'][time:time+time_delta])
    actual_predictor.append(file1)
  data = np.concatenate(actual_predictor, axis = 1)
  #data = np.expand_dims(data, axis = 0)
  for longitude in lon:
    for latitude in lat:
      dataframe[time:time+time_delta, latitude, longitude, 0] = model.predict(data[0:time_delta, :, latitude, longitude])[:, 0]
  del file1
  del actual_predictor
  del data
  gc.collect()

np.save('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/labels/labels_1827', dataframe)

ValueError: ignored

In [7]:
model.predict(data[0:time_delta, :, latitude, longitude])[:, 0].shape

(100,)

In [8]:
dataframe[time:time+time_delta, latitude, longitude, 0].shape

(0,)

In [ ]:
model.predict(data[time:time+time_delta, :, latitude, longitude])[:,0].shape

(100,)

In [ ]:
dataframe[time:time+time_delta, latitude, longitude, 0].shape

(100,)

In [ ]:
time_range = np.arange(0, 1827, 100)

In [ ]:
dataframe.shape

(1827, 285, 245, 1)

In [ ]:
dataframe[1800+26, :, :, 0].shape

(285, 245)

In [ ]:
file1.shape

(100, 20, 285, 245)

In [ ]:
data.shape

(100, 300, 285, 245)

In [ ]:
one = np.ma.getdata(nc.Dataset('/content/gdrive/MyDrive/rainfall-forecast/merge_model/populate_grid/ERA5/pressure_levels_prediction_preprocess/' + predictor)['__xarray_dataarray_variable__'][:])

In [ ]:
dataframe.shape

(1827, 285, 245, 1)

In [ ]:
data.shape

(1, 300, 285, 245)

In [ ]:
model.predict(data[time:time+1, latitude, longitude])

ValueError: ignored

In [ ]:
data[time:time+1, :, latitude, longitude]

array([], shape=(0, 300), dtype=float32)

In [ ]:
data.shape

(1, 300, 285, 245)

In [ ]:
t

2

In [ ]:
time_range

array([   0,    1,    2, ..., 1804, 1805, 1806])